In [ ]:
# librairies
import pandas as pd
import numpy as np 
import string 
import sys
import os 

# for visualization 
# Plot confusion matrix
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix
from matplotlib.pyplot import figure
import seaborn as sn

In [ ]:
!pip install gensim
!pip install keras

import re
import gensim
import nltk
import keras
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from keras.models import Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM, CuDNNGRU
from keras.layers import Dense, Embedding, LSTM, GRU, Bidirectional, SpatialDropout1D, SimpleRNN
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, MaxPooling2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.models import Model
from keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/Colab

## Processing the data 


In [ ]:
# import the dataset

# Path Kodjo
# path = '/home/anselme/Desktop/Etude/MVA_S1/DL/MVA_DL/Data Mining'

dataset = pd.read_excel('dataset.xlsx', skiprows=0)

In [ ]:
dataset['Class'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
from sklearn.utils import shuffle
dataset = shuffle(dataset)

dataset

In [ ]:
# Process lyrics
dataset["Lyrics"] = dataset["Lyrics"].str.replace('chorus','')

# Lyrics
Lyrics = [dataset['Lyrics'][i] for i in range(dataset.shape[0])]

# labels 
yData  = [dataset['Class'][i]  for i in range(dataset.shape[0])]


# remove text between [] (corresponds to Chrous / verse headings)
Lyrics = [re.sub("\[.*?\]", "",Lyrics[i]) for i in range( dataset.shape[0])]

# Clean up data (replace \n, [, (, ], ) par "")
Lyrics = [Lyrics[i].replace('\n', ' ') for i in  range( dataset.shape[0])]
Lyrics = [Lyrics[i].replace('[' ,  '') for i in  range( dataset.shape[0])]
Lyrics = [Lyrics[i].replace(']' ,  '') for i in  range( dataset.shape[0])]
Lyrics = [Lyrics[i].replace('(' ,  '') for i in  range( dataset.shape[0])]
Lyrics = [Lyrics[i].replace(')' ,  '') for i in  range( dataset.shape[0])]

#for i in range(100):
#  print(Lyrics[i])

xData = []
stop_words = set(stopwords.words('english'))
translator = str.maketrans('', '', string.punctuation)

for Lyric in Lyrics:
    # Tokenize each lyric, and set all characters to lower-case
    tokens = word_tokenize(Lyric)
    tokens = [word.lower() for word in tokens]
    
    # Remove punctuation
    tokens_nopunc = [word.translate(translator) for word in tokens]
    
    # Remove non-alphabetic tokens
    words = [word for word in tokens_nopunc if word.isalpha()]
    
    # Remove stop words from the lyric
    words = [word for word in words if not word in stop_words]
    
    # Append to training data
    xData.append(words)

for i in range(10):
  print(xData[i])

In [ ]:
# Loads pre-trained GloVe word embeddings

# Load in GloVe file and initialize embedding index
filename = 'wiki-news-300d-1M.vec'
file = open(os.path.join('', filename), encoding = "utf-8")
embeddings_index = {}

for line in file:

    # Add each embedding to the embedding index
    embedding = line.split()
    embeddings_index[embedding[0]] = np.asarray(embedding[1:])

file.close()

In [ ]:
# Map each word token in the training data to an integer

# For each training example, maps each word token to an integer
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(xData)
xData_seq = tokenizer.texts_to_sequences(xData)

In [ ]:
# Pad sequences shorter than max length 
max_num_tokens = max([len(tokenized_lyric) for tokenized_lyric in xData])
print(max_num_tokens)
xData_seq_padded = pad_sequences(xData_seq, maxlen=max_num_tokens)

In [ ]:
# Map GloVe word embeddings to each word in the tokenizer word index to create a matrix of word embeddings

# Initialize embedding matrix
EMBEDDING_DIM = 300
word_index = tokenizer.word_index
num_words = len(word_index) + 1

print(num_words)

embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

num = 0
# Populate embedding matrix
for word, i in word_index.items():
    
    if i > num_words:
        #print(word)
        num = num + 1
        continue
        
    # Assign corresponding GloVe embedding to the given word
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        # Words not found in the embedding are assigned a zero vector by default
        embedding_matrix[i] = embedding_vector
        
        if (all(abs(float(v)) < 0.0000000001 for v in embedding_vector)):
            #print(word)
            num = num + 1
            
    else:
        #print(word)
        num = num + 1
        
print(num)

In [ ]:
flat_list = [item for sublist in xData for item in sublist]

print(len(flat_list))

In [ ]:
# Shuffles the data, and splits it into train and test sets
np.random.seed(0)

VALIDATION_SPLIT = 0.1
word_indices = np.arange(xData_seq_padded.shape[0])
np.random.shuffle(word_indices)
xData_seq_padded = xData_seq_padded[word_indices]
mood = np.array(yData)
mood = mood[word_indices]

# Binarizes the mood labels
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
moods = encoder.fit_transform(mood.tolist())

# Splits the dataset into train and test sets
num_validation_samples = int(VALIDATION_SPLIT * xData_seq_padded.shape[0])

X_train_pad = xData_seq_padded[:-(2 * num_validation_samples)]
y_train = moods[:-(2 * num_validation_samples)]

X_val_pad = xData_seq_padded[-(2 * num_validation_samples):-num_validation_samples]
y_val = moods[-(2 * num_validation_samples):-num_validation_samples]

X_test_pad = xData_seq_padded[-num_validation_samples:]
y_test = moods[-num_validation_samples:]

print("Train set length: {}".format(len(X_train_pad)))
print("Validation set length: {}".format(len(X_val_pad)))
print("Test set length: {}".format(len(X_test_pad)))

In [ ]:
# Defines the Multiclass model, compiles and trains it
from keras.layers import add

model = Sequential()
embedding_layer = Embedding(num_words, 
                            EMBEDDING_DIM, 
                            embeddings_initializer=Constant(embedding_matrix), 
                            input_length=max_num_tokens,
                            trainable=False)

model.add(embedding_layer)
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(48, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(12, activation='softmax'))

checkpoint_filepath = 'checkpoint1'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_freq='epoch',
    verbose=1)

op = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer = op, loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

# Saves a History object for plotting
history = model.fit(X_train_pad, y_train, batch_size=16, epochs=40, 
                    validation_data=(X_val_pad, y_val), verbose=2, callbacks=[model_checkpoint_callback])

In [ ]:
# Plot accuracy as a function of training epoch

figure(num=None, figsize=(5, 5), dpi=300)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.grid(True)
plt.title('CNN with Fasttext Embeddings')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
model.load_weights(checkpoint_filepath)

results = model.evaluate(X_test_pad, y_test, batch_size=16)
print("test loss, test acc:", results)

print(model.summary())

In [ ]:
# Plot confusion matrix
alphabetical = []

for el in mood:
    if (el not in alphabetical):
        alphabetical.append(el)

alphabetical = sorted(alphabetical)

# Gets predicted labels from model
y_pred = model.predict(X_test_pad)

# Formats labels into appropriate class designations
y_pred_classes = np.zeros(len(y_pred)).astype(int)
y_test_classes = np.zeros(len(y_test)).astype(int)

for index, label in enumerate(y_pred):
    y_pred_classes[index] = np.argmax(y_pred[index])
    
for index, label in enumerate(y_test):
    y_test_classes[index] = np.argmax(y_test[index])

# Generates confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)

# Formats and displays the confusion matrix
figure(num=None, figsize=(5, 5), dpi=300)
df_cm = pd.DataFrame(cm, index = alphabetical, columns = alphabetical)
sn.heatmap(df_cm, annot=True, annot_kws={"size": 10}, cmap=plt.cm.Blues, fmt='g', cbar=False)
plt.title('CNN with Fasttext Embeddings Confusion Matrix', fontsize=9)
plt.xlabel("Predicted Class", fontsize=9)
plt.ylabel("Actual Class", fontsize=9)
plt.show()

In [ ]:
# Cell for getting information about each class

from sklearn.metrics import classification_report

new_test_classes = np.full(shape=y_test_classes.shape[0], fill_value="yoooooooooooooooo")
new_pred_classes = np.full(shape=y_pred_classes.shape[0], fill_value="yoooooooooooooooo")

for x in range(len(y_test_classes)):
    id1 = y_test_classes[x]
    id2 = y_pred_classes[x]

    word1 = alphabetical[id1]
    word2 = alphabetical[id2]

    new_test_classes[x] = word1
    new_pred_classes[x] = word2

print(classification_report(new_test_classes, new_pred_classes))

In [ ]:
# Defines the Multiclass model, compiles and trains it

lstm = Sequential()
embedding_layer = Embedding(num_words, 
                            EMBEDDING_DIM, 
                            embeddings_initializer=Constant(embedding_matrix), 
                            input_length=max_num_tokens,
                            trainable=False)

lstm.add(embedding_layer)
lstm.add(SpatialDropout1D(0.2))
lstm.add(Bidirectional(CuDNNLSTM(100)))
lstm.add(Dense(48, activation='relu'))
lstm.add(Dense(24, activation='relu'))
lstm.add(Dense(12, activation='softmax'))

checkpoint_filepath = 'LSTM1'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_freq='epoch',
    verbose=1)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.9)

op = keras.optimizers.Adam(learning_rate = lr_schedule)
lstm.compile(optimizer = op, loss='categorical_crossentropy', metrics=['accuracy'])

print(lstm.summary())

# Saves a History object for plotting
history = lstm.fit(X_train_pad, y_train, batch_size=128, epochs=40, 
                    validation_data=(X_val_pad, y_val), verbose=2, callbacks=[model_checkpoint_callback])

In [ ]:
# Plot accuracy as a function of training epoch

figure(num=None, figsize=(5, 5), dpi=300)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.grid(True)
plt.title('LSTM with Fasttext Embeddings')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
lstm.load_weights(checkpoint_filepath)

results = lstm.evaluate(X_test_pad, y_test, batch_size=128)
print("test loss, test acc:", results)

print(lstm.summary())

In [ ]:
# Plot confusion matrix
alphabetical = []

for el in mood:
    if (el not in alphabetical):
        alphabetical.append(el)

alphabetical = sorted(alphabetical)

# Gets predicted labels from model
y_pred = lstm.predict(X_test_pad)

# Formats labels into appropriate class designations
y_pred_classes = np.zeros(len(y_pred)).astype(int)
y_test_classes = np.zeros(len(y_test)).astype(int)

for index, label in enumerate(y_pred):
    y_pred_classes[index] = np.argmax(y_pred[index])
    
for index, label in enumerate(y_test):
    y_test_classes[index] = np.argmax(y_test[index])

# Generates confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)

# Formats and displays the confusion matrix
figure(num=None, figsize=(5, 5), dpi=300)
df_cm = pd.DataFrame(cm, index = alphabetical, columns = alphabetical)
sn.heatmap(df_cm, annot=True, annot_kws={"size": 10}, cmap=plt.cm.Blues, fmt='g', cbar=False)
plt.title('LSTM with Fasttext Embeddings Confusion Matrix', fontsize=9)
plt.xlabel("Predicted Class", fontsize=9)
plt.ylabel("Actual Class", fontsize=9)
plt.show()

In [ ]:
# Cell for getting information about each class

new_test_classes = np.full(shape=y_test_classes.shape[0], fill_value="yoooooooooooooooo")
new_pred_classes = np.full(shape=y_pred_classes.shape[0], fill_value="yoooooooooooooooo")

for x in range(len(y_test_classes)):
    id1 = y_test_classes[x]
    id2 = y_pred_classes[x]

    word1 = alphabetical[id1]
    word2 = alphabetical[id2]

    new_test_classes[x] = word1
    new_pred_classes[x] = word2

print(classification_report(new_test_classes, new_pred_classes))

In [ ]:
# Defines the Multiclass model, compiles and trains it

gru = Sequential()
embedding_layer = Embedding(num_words, 
                            EMBEDDING_DIM, 
                            embeddings_initializer=Constant(embedding_matrix), 
                            input_length=max_num_tokens,
                            trainable=False)

gru.add(embedding_layer)
gru.add(SpatialDropout1D(0.2))
gru.add(Bidirectional(CuDNNGRU(100)))
gru.add(Dense(48, activation='relu'))
gru.add(Dense(24, activation='relu'))
gru.add(Dense(12, activation='softmax'))

checkpoint_filepath = 'GRU1'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_freq='epoch',
    verbose=1)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.9)

op = keras.optimizers.Adam(learning_rate = lr_schedule)
gru.compile(optimizer = op, loss='categorical_crossentropy', metrics=['accuracy'])

print(gru.summary())

# Saves a History object for plotting
history = gru.fit(X_train_pad, y_train, batch_size=128, epochs=40, 
                    validation_data=(X_val_pad, y_val), verbose=2, callbacks=[model_checkpoint_callback])

In [ ]:
# Plot accuracy as a function of training epoch

figure(num=None, figsize=(5, 5), dpi=300)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.grid(True)
plt.title('GRU with Fasttext Embeddings')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
gru.load_weights(checkpoint_filepath)

results = gru.evaluate(X_test_pad, y_test, batch_size=128)
print("test loss, test acc:", results)

print(gru.summary())

In [ ]:
# Plot confusion matrix
alphabetical = []

for el in mood:
    if (el not in alphabetical):
        alphabetical.append(el)

alphabetical = sorted(alphabetical)

# Gets predicted labels from model
y_pred = gru.predict(X_test_pad)

# Formats labels into appropriate class designations
y_pred_classes = np.zeros(len(y_pred)).astype(int)
y_test_classes = np.zeros(len(y_test)).astype(int)

for index, label in enumerate(y_pred):
    y_pred_classes[index] = np.argmax(y_pred[index])
    
for index, label in enumerate(y_test):
    y_test_classes[index] = np.argmax(y_test[index])

# Generates confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)

# Formats and displays the confusion matrix
figure(num=None, figsize=(5, 5), dpi=300)
df_cm = pd.DataFrame(cm, index = alphabetical, columns = alphabetical)
sn.heatmap(df_cm, annot=True, annot_kws={"size": 10}, cmap=plt.cm.Blues, fmt='g', cbar=False)
plt.title('GRU with Fasttext Embeddings Confusion Matrix', fontsize=9)
plt.xlabel("Predicted Class", fontsize=9)
plt.ylabel("Actual Class", fontsize=9)
plt.show()